# Multi‑Qubit Chain with ARP Dynamics (Minimal Kit)

This notebook simulates an N‑qubit XY chain with ARP feedback on each link:
$\dot G=\alpha|I(\psi)|-\mu G$, and a parametric drive with index β.

In [ ]:
from importlib import util
spec=util.spec_from_file_location('quantum_chain','quantum_chain.py')
qc=util.module_from_spec(spec); spec.loader.exec_module(qc)

In [ ]:
# quick run
import numpy as np, matplotlib.pyplot as plt
from mpmath import besselj

def run(N=4,beta=0.15,alpha=2.0,mu=50.0,eta_mean=0.04,tmax=3.5e-7,dt=1e-9):
    Zs,Hl=qc.build_static_terms(N)
    Sps=[qc.op_at(i,qc.Sp,N) for i in range(N)]; Sms=[qc.op_at(i,qc.Sm,N) for i in range(N)]
    t=np.arange(0,tmax,dt); psi=np.zeros(2**N,dtype=complex); psi[1<<(N-1)]=1; psi/=np.linalg.norm(psi)
    G=np.zeros(N-1); m=float(2*besselj(1,beta)); g0=50e6*2*np.pi; w0=6e9*2*np.pi
    pops=np.zeros((N,len(t)))
    for k,tt in enumerate(t):
        omegas=np.ones(N)*w0*(1+eta_mean)
        gs=np.ones(N-1)*g0*(1+m*np.sin(2*np.pi*300e3*tt))*(1+0.2*G)
        H=qc.hamiltonian(N,Zs,Hl,omegas,gs)
        w,V=np.linalg.eigh(H); psi=(V@np.diag(np.exp(-1j*w*(dt)))@V.conj().T)@psi
        pops[:,k]=qc.populations(psi,N)
        I=qc.link_current_expectation(psi,Sps,Sms); G=np.clip(G+dt*(alpha*I-mu*G),0,1)
    return t,pops

T,P=run(); plt.imshow(P,aspect='auto',origin='lower',extent=[T[0]*1e6,T[-1]*1e6,0,P.shape[0]-1]);
plt.xlabel('time (μs)'); plt.ylabel('site'); plt.colorbar(label='P'); plt.show()